In [1]:
from numpy import array
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, KFold

from sklearn.base import clone

import time

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
columns = ['cases_new','cases_active','checkins','percent_vax', 'temp']
target = 'cases_new'
# choose a number of time steps
n_steps = 5
n_features = len(columns)
state = 11

In [4]:
df_new = pd.read_csv('train_states.csv', index_col=0)
df_new = df_new.sort_values(by = 'date')
#X = X.reshape((X.shape[0], X.shape[1], n_features))
df_new

,date,cases_new,state_id,cases_recovered,cases_death,cases_active,checkins,unique_ind,cumul_full,pop,percent_vax,temp
0,2021-02-24,318,2,518,0,6156,2074008,927705,0,3781000,0.00,25.80
15,2021-02-24,1,17,5,0,51,195780,113866,0,110000,0.00,28.40
14,2021-02-24,5,16,1,0,22,69982,28861,0,99600,0.00,26.20
13,2021-02-24,381,15,432,7,2711,3562969,1647182,0,1773700,0.00,29.40
12,2021-02-24,22,12,63,1,297,354767,175197,0,1259300,0.00,25.95
...,...,...,...,...,...,...,...,...,...,...,...,...
4865,2021-12-25,192,3,184,0,1644,1182090,605241,1568408,2185100,0.72,28.90
4864,2021-12-25,416,2,1232,3,5188,3603159,1506124,3036641,3781000,0.80,27.70
4878,2021-12-25,2,16,10,0,62,71513,28947,79555,99600,0.80,27.70
4870,2021-12-25,164,9,0,4,1899,2093567,915909,1852221,2510300,0.74,27.80


In [5]:
df = df_new[df_new['state_id']==state]
df

,date,cases_new,state_id,cases_recovered,cases_death,cases_active,checkins,unique_ind,cumul_full,pop,percent_vax,temp
11,2021-02-24,581,11,1322,2,10490,4302048,2070162,0,6538000,0.00,28.90
27,2021-02-25,666,11,1604,9,9551,4385034,2115153,0,6538000,0.00,28.25
43,2021-02-26,633,11,1149,4,9034,4655887,2209354,0,6538000,0.00,28.95
59,2021-02-27,862,11,1132,2,8763,5315892,2313602,0,6538000,0.00,29.20
75,2021-02-28,914,11,943,3,8731,5101079,2227024,1,6538000,0.00,28.15
...,...,...,...,...,...,...,...,...,...,...,...,...
4811,2021-12-21,736,11,1172,9,14104,6951905,3221224,4755638,6538000,0.73,29.05
4827,2021-12-22,945,11,1414,4,13635,7126497,3269055,4756874,6538000,0.73,27.25
4843,2021-12-23,951,11,1428,6,13154,7195693,3299399,4758165,6538000,0.73,27.70
4859,2021-12-24,848,11,1263,3,12739,7792725,3467531,4759718,6538000,0.73,28.75


In [6]:
X = df[columns]
X

,cases_new,cases_active,checkins,percent_vax,temp
11,581,10490,4302048,0.00,28.90
27,666,9551,4385034,0.00,28.25
43,633,9034,4655887,0.00,28.95
59,862,8763,5315892,0.00,29.20
75,914,8731,5101079,0.00,28.15
...,...,...,...,...,...
4811,736,14104,6951905,0.73,29.05
4827,945,13635,7126497,0.73,27.25
4843,951,13154,7195693,0.73,27.70
4859,848,12739,7792725,0.73,28.75


In [7]:
# normalize encoder
X_scaler = MinMaxScaler()
X_scaler.fit(X)

y_scaler = MinMaxScaler()
y_scaler.fit(array(df[target]).reshape(-1, 1))

X = X_scaler.transform(X)
X = pd.DataFrame(X, columns = columns)
X

,cases_new,cases_active,checkins,percent_vax,temp
0,0.042226,0.067006,0.492313,0.0,0.811321
1,0.052140,0.056533,0.503417,0.0,0.688679
2,0.048291,0.050767,0.539658,0.0,0.820755
3,0.075003,0.047745,0.627970,0.0,0.867925
4,0.081068,0.047388,0.599227,0.0,0.669811
...,...,...,...,...,...
300,0.060306,0.107312,0.846874,1.0,0.839623
301,0.084684,0.102081,0.870236,1.0,0.500000
302,0.085384,0.096717,0.879494,1.0,0.584906
303,0.073370,0.092088,0.959379,1.0,0.783019


In [8]:
for i in range(n_steps):
    shifted_X = X.shift(i+1)
    X['cn_'+str(i+1)] = shifted_X['cases_new']
    X['ca_'+str(i+1)] = shifted_X['cases_active']
    X['ci_'+str(i+1)] = shifted_X['checkins']
    X['pv_'+str(i+1)] = shifted_X['percent_vax']
    X['t_'+str(i+1)] = shifted_X['temp']
    
    #df['ui_'+str(i+1)] = shifted_df['unique_ind']
X = X.dropna()

In [9]:
y = X[target]

In [10]:
cols = []
features = ['cn', 'ca', 'ci', 'pv', 't']

for i in range(n_steps):
    for f in features:
        cols.append(f+"_"+str(i+1))
cols

['cn_1',
 'ca_1',
 'ci_1',
 'pv_1',
 't_1',
 'cn_2',
 'ca_2',
 'ci_2',
 'pv_2',
 't_2',
 'cn_3',
 'ca_3',
 'ci_3',
 'pv_3',
 't_3',
 'cn_4',
 'ca_4',
 'ci_4',
 'pv_4',
 't_4',
 'cn_5',
 'ca_5',
 'ci_5',
 'pv_5',
 't_5']

In [11]:
X = X[cols]
X

,cn_1,ca_1,ci_1,pv_1,t_1,cn_2,ca_2,ci_2,pv_2,t_2,...,cn_4,ca_4,ci_4,pv_4,t_4,cn_5,ca_5,ci_5,pv_5,t_5
5,0.081068,0.047388,0.599227,0.0,0.669811,0.075003,0.047745,0.627970,0.0,0.867925,...,0.052140,0.056533,0.503417,0.0,0.688679,0.042226,0.067006,0.492313,0.0,0.811321
6,0.027295,0.041198,0.534913,0.0,0.811321,0.081068,0.047388,0.599227,0.0,0.669811,...,0.048291,0.050767,0.539658,0.0,0.820755,0.052140,0.056533,0.503417,0.0,0.688679
7,0.052840,0.038555,0.516739,0.0,0.613208,0.027295,0.041198,0.534913,0.0,0.811321,...,0.075003,0.047745,0.627970,0.0,0.867925,0.048291,0.050767,0.539658,0.0,0.820755
8,0.049108,0.037919,0.538098,0.0,0.688679,0.052840,0.038555,0.516739,0.0,0.613208,...,0.081068,0.047388,0.599227,0.0,0.669811,0.075003,0.047745,0.627970,0.0,0.867925
9,0.047941,0.036938,0.540055,0.0,0.858491,0.049108,0.037919,0.538098,0.0,0.688679,...,0.027295,0.041198,0.534913,0.0,0.811321,0.081068,0.047388,0.599227,0.0,0.669811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,0.018780,0.112241,0.833844,1.0,0.688679,0.081652,0.117862,0.770331,1.0,0.320755,...,0.142541,0.123171,0.836097,1.0,0.075472,0.136358,0.125703,0.882397,1.0,0.405660
301,0.060306,0.107312,0.846874,1.0,0.839623,0.018780,0.112241,0.833844,1.0,0.688679,...,0.104398,0.120505,0.692208,1.0,0.000000,0.142541,0.123171,0.836097,1.0,0.075472
302,0.084684,0.102081,0.870236,1.0,0.500000,0.060306,0.107312,0.846874,1.0,0.839623,...,0.081652,0.117862,0.770331,1.0,0.320755,0.104398,0.120505,0.692208,1.0,0.000000
303,0.085384,0.096717,0.879494,1.0,0.584906,0.084684,0.102081,0.870236,1.0,0.500000,...,0.018780,0.112241,0.833844,1.0,0.688679,0.081652,0.117862,0.770331,1.0,0.320755


In [12]:
X = np.array(X).reshape((X.shape[0], n_steps, n_features))
X

array([[[0.08106847, 0.04738803, 0.5992268 , 0.        , 0.66981132],
        [0.07500292, 0.04774491, 0.62796961, 0.        , 0.86792453],
        [0.04829115, 0.05076731, 0.53965839, 0.        , 0.82075472],
        [0.05214044, 0.05653328, 0.50341722, 0.        , 0.68867925],
        [0.04222559, 0.06700571, 0.49231337, 0.        , 0.81132075]],

       [[0.027295  , 0.04119825, 0.53491331, 0.        , 0.81132075],
        [0.08106847, 0.04738803, 0.5992268 , 0.        , 0.66981132],
        [0.07500292, 0.04774491, 0.62796961, 0.        , 0.86792453],
        [0.04829115, 0.05076731, 0.53965839, 0.        , 0.82075472],
        [0.05214044, 0.05653328, 0.50341722, 0.        , 0.68867925]],

       [[0.05284031, 0.03855505, 0.51673873, 0.        , 0.61320755],
        [0.027295  , 0.04119825, 0.53491331, 0.        , 0.81132075],
        [0.08106847, 0.04738803, 0.5992268 , 0.        , 0.66981132],
        [0.07500292, 0.04774491, 0.62796961, 0.        , 0.86792453],
        [0.04829

In [13]:
y = array(y).reshape(-1, 1)
y

array([[0.027295  ],
       [0.05284031],
       [0.04910766],
       [0.04794121],
       [0.06707104],
       [0.07080369],
       [0.06147206],
       [0.05913916],
       [0.02857809],
       [0.03989269],
       [0.05750612],
       [0.06205529],
       [0.03872623],
       [0.04175901],
       [0.01866325],
       [0.01213111],
       [0.03266068],
       [0.01563047],
       [0.01504724],
       [0.05050741],
       [0.0159804 ],
       [0.02146273],
       [0.04735798],
       [0.03161087],
       [0.0159804 ],
       [0.03021113],
       [0.0074653 ],
       [0.00431588],
       [0.        ],
       [0.02122944],
       [0.05155721],
       [0.00209962],
       [0.02752829],
       [0.01551382],
       [0.01446401],
       [0.01259769],
       [0.00268284],
       [0.0087484 ],
       [0.01808002],
       [0.02146273],
       [0.02286248],
       [0.03685991],
       [0.0097982 ],
       [0.03079435],
       [0.03476029],
       [0.02799487],
       [0.0432754 ],
       [0.062

In [32]:
# define model
def create_model():
    model = Sequential()
    model.add(LSTM(1000, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [33]:
print('length X: ', len(X))
print('length y: ', len(y))

length X:  300
length y:  300


In [34]:
models = []
scores = []
#tscv = TimeSeriesSplit(n_splits=10)
kf = KFold(n_splits=10)
t0 = time.time()

for train_index, test_index in kf.split(X):
    model_new = create_model()
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    t1 = time.time()
    # fit model
    model_new.fit(X_train, y_train, epochs=len(X), verbose=0)
    t_total = time.time()-t1
    print("time-taken: ", int(t_total/60), "mins", int(t_total%60), "seconds", int(t_total*1000%1000), "miliseconds")

    
    y_test_ = y_scaler.inverse_transform(y_test)
    y_pred_ = y_scaler.inverse_transform(model_new.predict(X_test))
    result = mean_squared_error(y_test_, y_pred_)**0.5

    models.append(model_new)
    scores.append(result)

time-taken:  3 mins 53 seconds 322 miliseconds
time-taken:  4 mins 8 seconds 759 miliseconds
time-taken:  4 mins 0 seconds 429 miliseconds
time-taken:  4 mins 7 seconds 998 miliseconds
time-taken:  4 mins 9 seconds 261 miliseconds
time-taken:  4 mins 11 seconds 380 miliseconds
time-taken:  4 mins 3 seconds 514 miliseconds
time-taken:  4 mins 9 seconds 268 miliseconds
time-taken:  4 mins 7 seconds 163 miliseconds
time-taken:  4 mins 8 seconds 672 miliseconds


In [35]:
t_total = time.time()-t0
print("time-taken: ", int(t_total/60), "mins", int(t_total%60), "seconds", int(t_total*1000%1000), "miliseconds")

time-taken:  41 mins 13 seconds 671 miliseconds


In [18]:
scores

[216.95558293841054,
 141.36849941402846,
 434.19857761873914,
 993.707235883258,
 1131.2336197322088,
 1253.1413016929073,
 1591.4115165364144,
 535.2827542099992,
 246.59232840920444,
 376.9021452976014]

In [36]:
scores

[336.9993785138915,
 181.06771371784143,
 444.2690758826869,
 990.3202235413506,
 1009.5148689753036,
 1301.3914867893375,
 1624.2040524769627,
 381.17047750542156,
 204.3171455849426,
 308.17330068064865]